This is a PEI Assessment Ingestion Transformation Load

We have ingested the data, the data found to be in all three different formats Configurations are uploaded to make the parsing and date conversion in Spark version >= 3.0 We have done bunch of transformations : - missing nulls hence done their removal and showed original and new count cleaning of data incuding phone performing cleaning and make it normalized column such as phone Performed join operation However I assume the pyspark queries needs to be translated back to spark sql queries.

In [216]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName('pei-assessment')\
    .config('spark.sql.shufflePartitons',3)\
    .enableHiveSupport() \
    .master('local[*]')\
    .getOrCreate()

In [217]:
spark

In [218]:
from pyspark.sql.functions import * 
from pyspark.sql.window import *


##  installation of packages 

pip install openpyxl
import findspark
findspark.init()

import pyspark
pip install spark-excel
!pip install pandas
!pip install pyarrow





In [219]:
import pandas as pd


pandasDF = pd.read_excel(io = 'Customer.xlsx', engine='openpyxl', sheet_name = 'Worksheet') 

In [220]:
pandasDF.head(3)

,Customer ID,Customer Name,email,phone,address,Segment,Country,City,State,Postal Code,Region
0,PW-19240,Pierre Wener,bettysullivan808@gmail.com,421.580.0902x9815,"001 Jones Ridges Suite 338\nJohnsonfort, FL 95462",Consumer,United States,Louisville,Colorado,80027,West
1,GH-14410,Gary567 Hansen,austindyer948@gmail.com,001-542-415-0246x314,"00347 Murphy Unions\nAshleyton, IA 29814",Home Office,United States,Chicago,Illinois,60653,Central
2,KL-16555,Kelly Lampkin,clarencehughes280@gmail.com,7185624866,"007 Adams Lane Suite 176\nEast Amyberg, IN 34581",Corporate,United States,Colorado Springs,Colorado,80906,West


In [221]:
#pandasDF=pandasDF.astype(str)

In [222]:
pandasDF.dtypes

Customer ID      object
Customer Name    object
email            object
phone            object
address          object
Segment          object
Country          object
City             object
State            object
Postal Code       int64
Region           object
dtype: object

In [223]:
customer_schema= 'customer_id string , customer_name string , email string ,phone string ,address string , segment string,country string ,city string , state string , postal_code string , region string'

In [224]:
pandasDF.rename(columns={'Customer ID':'customer_id', 'Customer Name':'customer_name', 'Country':'country','City':'city','State':'state','Postal Code':'postal_code','Region':'region'},inplace=True)

In [225]:
pandasDF = pandasDF.reset_index(drop=True)

In [226]:
csv_data = pandasDF.to_csv('customer.csv', index = False)

In [227]:
len(pandasDF)

793

In [228]:
customer_df=spark.read\
    .format('csv')\
    .option('header',True)\
    .option('inferSchema',True)\
    .schema(customer_schema)\
    .load('customer.csv')

Found discrepancy Checking in converted customer dataframe from pandas df hence checking additional row and cleaning

In [229]:
import unittest

def df_count_match(value):
    print(f'checking whether {value} is equals to orignal pandas datafram count {len(pandasDF)} ')
    return value > len(pandasDF)

# Test class to test the is_positive function
class CustomerDataFrameCheck(unittest.TestCase):
    # Test case for a positive value
    def test_positive_value(self):
        expected_result=False
        #self.assertTrue(is_positive(10))
        actual_result = df_count_match(customer_df.count())
        print(f'{actual_result} and {actual_result}')
        self.assertEqual(expected_result, actual_result)
    def reformattingdataframeandchecking(self):
        expected_result=True
        #self.assertTrue(is_positive(10))
        actual_result = df_count_match(customer_df.count())
        print(f'{actual_result} and {actual_result}')
        self.assertEqual(expected_result, actual_result)
        

In [230]:
# Run the test cases
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(CustomerDataFrameCheck))

F
FAIL: test_positive_value (__main__.CustomerDataFrameCheck)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\vineetjain8\AppData\Local\Temp\ipykernel_5624\3913042169.py", line 15, in test_positive_value
    self.assertEqual(expected_result, actual_result)
AssertionError: False != True

----------------------------------------------------------------------
Ran 1 test in 0.112s

FAILED (failures=1)


checking whether 1586 is equals to orignal pandas datafram count 793 
True and True


<unittest.runner.TextTestResult run=1 errors=0 failures=1>


### found redundant count appeard while comparing with pandas dataframe

In [285]:
customer_df =customer_df.filter(col('customer_id').rlike('-'))

In [232]:
import unittest

def df_count_match(value):
    print(f'checking whether {value} is equals to orignal pandas datafram count {len(pandasDF)} ')
    return value == len(pandasDF)

# Test class to test the is_positive function
class CustomerDataFrameCheck(unittest.TestCase):
    # Test case for a positive value
    def test_positive_value(self):
        expected_result=True
        #self.assertTrue(is_positive(10))
        actual_result = df_count_match(customer_df.count())
        print(f'{actual_result} and {actual_result}')
        self.assertEqual(expected_result, actual_result)
        

In [233]:
# Run the test cases
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(CustomerDataFrameCheck))

.
----------------------------------------------------------------------
Ran 1 test in 0.123s

OK


checking whether 793 is equals to orignal pandas datafram count 793 
True and True


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [234]:
order_df =spark.read.format('json').option('header',True)\
            .option('inferSchema',True)\
            .option('multiline',True)\
    .load('Order.json')

In [235]:
product_schema = 'product_id string, category string,sub_category string, product_name string,state string, price_per_product float'


In [236]:
product_df = spark.read\
        .format('csv')\
            .option('header',True)\
                    .schema(product_schema)\
                    .load('Product.csv')

In [237]:
product_df.show(2)

+---------------+----------+------------+--------------------+--------+-----------------+
|     product_id|  category|sub_category|        product_name|   state|price_per_product|
+---------------+----------+------------+--------------------+--------+-----------------+
|FUR-CH-10002961| Furniture|      Chairs|Leather Task Chai...|New York|           81.882|
|TEC-AC-10004659|Technology| Accessories|Imation Secure+ H...|Oklahoma|            72.99|
+---------------+----------+------------+--------------------+--------+-----------------+
only showing top 2 rows



## Source DataFrames 
### customer_df, order_df , product_df

In [238]:
order_df = order_df.withColumnRenamed('Customer ID','customer_id')\
    .withColumnRenamed('Product ID','product_id')\
        .withColumnRenamed('Order Date','order_date')\
        .withColumnRenamed('Discount','discount')\
            .withColumnRenamed('Order ID','order_id')\
                .withColumnRenamed('Profit','profit')\
                    .withColumnRenamed('Quantity','quantity')\
                        .withColumnRenamed('Row ID','row_id')\
                            .withColumnRenamed('Ship Date','ship_date')\
                                .withColumnRenamed('Ship Mode','ship_mode')\





In [239]:
order_df.show(2)

+-----------+--------+----------+--------------+-------+---------------+-------+--------+------+---------+--------------+
|customer_id|discount|order_date|      order_id|  Price|     product_id| profit|quantity|row_id|ship_date|     ship_mode|
+-----------+--------+----------+--------------+-------+---------------+-------+--------+------+---------+--------------+
|   JK-15370|     0.3| 21/8/2016|CA-2016-122581|573.174|FUR-CH-10002961| 63.686|       7|     1|25/8/2016|Standard Class|
|   BD-11320|     0.0| 23/9/2017|CA-2017-117485| 291.96|TEC-AC-10004659|102.186|       4|     2|29/9/2017|Standard Class|
+-----------+--------+----------+--------------+-------+---------------+-------+--------+------+---------+--------------+
only showing top 2 rows



In [240]:
product_df.count()

1851

In [241]:
#product_df= product_df.isNull()

### Checking presence of null in Product Dataframe

In [242]:
product_new_Df = product_df.na.drop()

In [243]:
print(f'Original dataframe has product_df {product_df.count()} rows and  after removing null we have  product_new_Df with {product_new_Df.count()}  rows ') 


Original dataframe has product_df 1851 rows and  after removing null we have  product_new_Df with 1799  rows 


Profit rounded to 2 decimal places

In [244]:
date_format='yyyy-MM-dd'
order_df= order_df.withColumn('profit', round(col('profit'),2))\
        .withColumn('order_date',trim(col('order_date')))\
            .withColumn('ship_date',trim(col('order_date')))\

        

In [245]:
order_df.show(4)

+-----------+--------+----------+--------------+-------+---------------+------+--------+------+---------+--------------+
|customer_id|discount|order_date|      order_id|  Price|     product_id|profit|quantity|row_id|ship_date|     ship_mode|
+-----------+--------+----------+--------------+-------+---------------+------+--------+------+---------+--------------+
|   JK-15370|     0.3| 21/8/2016|CA-2016-122581|573.174|FUR-CH-10002961| 63.69|       7|     1|21/8/2016|Standard Class|
|   BD-11320|     0.0| 23/9/2017|CA-2017-117485| 291.96|TEC-AC-10004659|102.19|       4|     2|23/9/2017|Standard Class|
|   LB-16795|     0.7| 6/10/2016|US-2016-157490|     17|OFF-BI-10002824|-14.92|       4|     3|6/10/2016|   First Class|
|   KB-16315|     0.2|  2/7/2015|CA-2015-111703| 15.552|OFF-PA-10003349|  5.64|       3|     4| 2/7/2015|Standard Class|
+-----------+--------+----------+--------------+-------+---------------+------+--------+------+---------+--------------+
only showing top 4 rows



Checking presence of  characters such as ERRROR,.,( )  in phone columns of customers dataframe

In [246]:
import unittest

def df_error_match(value):
    print(f'checking presence of unknown characters  ')
    isErrorWordExists = customer_df.filter(col('phone').rlike('#ERROR!')).count()
    return 0 == isErrorWordExists

# Test class to test the is_positive function
class CustomerDataFrameCheck(unittest.TestCase):
    # Test case for a positive value
    def test_error_word(self):
        expected_result=False
        #self.assertTrue(is_positive(10))
        actual_result = df_error_match(customer_df.count())
        print(f'{actual_result} and {actual_result}')
        self.assertEqual(expected_result, actual_result)


In [247]:
# Run the test cases
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(CustomerDataFrameCheck))

.
----------------------------------------------------------------------
Ran 1 test in 0.182s

OK


checking presence of unknown characters  
False and False


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Enriching customer name 
Customer name and country


In [248]:
#customer_df_bkp = customer_df
customer_df =customer_df.filter(col('phone') !='#ERROR!').filter(lit(length(col('phone'))>lit(5))).withColumn('phone',regexp_replace('phone','x[0-9]+',''))\
        .withColumn('phone',regexp_replace('phone','[\\.\\-\\(\\)]',''))\
            .withColumn('phone',substring(col('phone'),1,10))
customer_df.show(10)


+-----------+-------------------+--------------------+----------+--------------------+-------+-------+----+-----+-----------+------+
|customer_id|      customer_name|               email|     phone|             address|segment|country|city|state|postal_code|region|
+-----------+-------------------+--------------------+----------+--------------------+-------+-------+----+-----+-----------+------+
|   PW-19240|       Pierre Wener|bettysullivan808@...|4215800902|001 Jones Ridges ...|   null|   null|null| null|       null|  null|
|   GH-14410|     Gary567 Hansen|austindyer948@gma...|0015424150| 00347 Murphy Unions|   null|   null|null| null|       null|  null|
|   KL-16555|      Kelly Lampkin|clarencehughes280...|7185624866|007 Adams Lane Su...|   null|   null|null| null|       null|  null|
|   AH-10075|Ad.       ..am Hart|angelabryant256@g...|2651015569|01454 Christopher...|   null|   null|null| null|       null|  null|
|   PF-19165|         Philip Fox|kristinereynolds5...|0014736452|0158

In [249]:
import unittest

def df_error_match(value):
    print(f'checking presence of unknown characters  ')
    isErrorWordExists = customer_df.filter(col('customer_name').rlike('[@.<>#$%^&*]')).count()
    return 0 == isErrorWordExists

# Test class to test the is_positive function
class CustomerNameDataFrameCheck(unittest.TestCase):
    # Test case for a positive value
    def test_error_word(self):
        expected_result=True
        #self.assertTrue(is_positive(10))
        actual_result = df_error_match(customer_df.count())
        print(f'{actual_result} and {actual_result}')
        self.assertEqual(expected_result, actual_result)


In [250]:
# Run the test cases
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(CustomerNameDataFrameCheck))

F
FAIL: test_error_word (__main__.CustomerNameDataFrameCheck)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\vineetjain8\AppData\Local\Temp\ipykernel_5624\2574361274.py", line 16, in test_error_word
    self.assertEqual(expected_result, actual_result)
AssertionError: True != False

----------------------------------------------------------------------
Ran 1 test in 0.199s

FAILED (failures=1)


checking presence of unknown characters  
False and False


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

In [251]:
#from pyspark.sql.functions import * 
customer_df = customer_df.withColumn('customer_name',trim('customer_name'))\
.withColumn('customer_name',regexp_replace('customer_name','[@.<>#$%^&*]',''))\
    .withColumn('customer_name',regexp_replace('customer_name','^\\s+|\\s+$',''))\
         .withColumn('customer_name',regexp_replace('customer_name','\\s+',' '))

      

#print(f'old customer_df count is {customer_df.count()} and new customer df count is {new_customer_df.count()}')

In [252]:
import unittest

def df_error_match(value):
    print(f'checking presence of unknown characters  ')
    isErrorWordExists = customer_df.filter(col('customer_name').rlike('[@.<>#$%^&*]')).count()
    return 0 == isErrorWordExists

# Test class to test the is_positive function
class CustomerNameDataFrameCheck(unittest.TestCase):
    # Test case for a positive value
    def test_error_word(self):
        expected_result=True
        #self.assertTrue(is_positive(10))
        actual_result = df_error_match(customer_df.count())
        print(f'{actual_result} and {actual_result}')
        self.assertEqual(expected_result, actual_result)


In [253]:
# Run the test cases
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(CustomerNameDataFrameCheck))

.
----------------------------------------------------------------------
Ran 1 test in 0.198s

OK


checking presence of unknown characters  
True and True


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Result Set from Question 1 

In [254]:
problem1_1 = order_df
problem1_2 = customer_df.select('customer_name','country')
problem1_3=product_new_Df.select('product_name','sub_category')

In [255]:
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [256]:
order_df = order_df.withColumn('order_date', to_date('order_date','dd/MM/yyyy'))\
        .withColumn('ship_date',to_date('ship_date','dd/MM/yyyy'))


In [257]:
order_df= order_df.withColumn('profit', col('profit').cast('float'))

In [258]:
order_df.show(4)

+-----------+--------+----------+--------------+-------+---------------+------+--------+------+----------+--------------+
|customer_id|discount|order_date|      order_id|  Price|     product_id|profit|quantity|row_id| ship_date|     ship_mode|
+-----------+--------+----------+--------------+-------+---------------+------+--------+------+----------+--------------+
|   JK-15370|     0.3|2016-08-21|CA-2016-122581|573.174|FUR-CH-10002961| 63.69|       7|     1|2016-08-21|Standard Class|
|   BD-11320|     0.0|2017-09-23|CA-2017-117485| 291.96|TEC-AC-10004659|102.19|       4|     2|2017-09-23|Standard Class|
|   LB-16795|     0.7|2016-10-06|US-2016-157490|     17|OFF-BI-10002824|-14.92|       4|     3|2016-10-06|   First Class|
|   KB-16315|     0.2|2015-07-02|CA-2015-111703| 15.552|OFF-PA-10003349|  5.64|       3|     4|2015-07-02|Standard Class|
+-----------+--------+----------+--------------+-------+---------------+------+--------+------+----------+--------------+
only showing top 4 rows


Create an aggregate table that shows profit by

Year

Product Category

Product Sub Category

Customer


In [259]:
order_df = order_df.withColumn('year', year(col('order_date')))


In [260]:
order_df.show(3)

+-----------+--------+----------+--------------+-------+---------------+------+--------+------+----------+--------------+----+
|customer_id|discount|order_date|      order_id|  Price|     product_id|profit|quantity|row_id| ship_date|     ship_mode|year|
+-----------+--------+----------+--------------+-------+---------------+------+--------+------+----------+--------------+----+
|   JK-15370|     0.3|2016-08-21|CA-2016-122581|573.174|FUR-CH-10002961| 63.69|       7|     1|2016-08-21|Standard Class|2016|
|   BD-11320|     0.0|2017-09-23|CA-2017-117485| 291.96|TEC-AC-10004659|102.19|       4|     2|2017-09-23|Standard Class|2017|
|   LB-16795|     0.7|2016-10-06|US-2016-157490|     17|OFF-BI-10002824|-14.92|       4|     3|2016-10-06|   First Class|2016|
+-----------+--------+----------+--------------+-------+---------------+------+--------+------+----------+--------------+----+
only showing top 3 rows



In [261]:
order_df.createOrReplaceTempView('orders')

In [262]:
profit_by_year = order_df.groupBy('year').agg(sum('profit').alias('total'))

In [263]:
customer_df= customer_df.withColumnRenamed('customer_id','customer__id')

In [264]:
customer_df.select('customer__id').show(3)

+------------+
|customer__id|
+------------+
|    PW-19240|
|    GH-14410|
|    KL-16555|
+------------+
only showing top 3 rows



In [265]:
customer_df.dtypes

[('customer__id', 'string'),
 ('customer_name', 'string'),
 ('email', 'string'),
 ('phone', 'string'),
 ('address', 'string'),
 ('segment', 'string'),
 ('country', 'string'),
 ('city', 'string'),
 ('state', 'string'),
 ('postal_code', 'string'),
 ('region', 'string')]

In [266]:
merge_df =order_df.join(customer_df ,order_df.customer_id==customer_df.customer__id , how='inner')\
        .join(product_new_Df, on='product_id',how='inner')

In [268]:
merge_df.groupBy('category').agg(round(sum('profit'),2).alias('cat_wise_sum')).show()

+---------------+------------+
|       category|cat_wise_sum|
+---------------+------------+
|Office Supplies|   114968.33|
|      Furniture|      3361.5|
|     Technology|   104157.16|
+---------------+------------+



In [269]:
merge_df.groupBy('sub_category').agg(round(sum('profit'),2).alias('sub_cat_wise_sum')).show()

+------------+----------------+
|sub_category|sub_cat_wise_sum|
+------------+----------------+
|   Envelopes|         5107.39|
|         Art|         5180.91|
|      Chairs|        21508.91|
| Furnishings|          129.33|
|    Supplies|        -1525.05|
|   Fasteners|          626.88|
|     Binders|        33692.77|
|   Bookcases|        -2900.87|
|      Labels|         4785.94|
|       Paper|        26986.03|
| Accessories|        40141.17|
|     Copiers|        47911.96|
|      Phones|        38156.26|
|    Machines|       -22052.23|
|     Storage|         19891.1|
|  Appliances|        20222.36|
|      Tables|       -15375.87|
+------------+----------------+



In [270]:
merge_df.groupBy('customer_id').agg(round(sum('profit'),2).alias('customerprofit')).show()

+-----------+--------------+
|customer_id|customerprofit|
+-----------+--------------+
|   VW-21775|       -882.39|
|   RR-19315|        -73.83|
|   PB-19210|          21.9|
|   EM-13960|         102.3|
|   MS-17530|         81.77|
|   KH-16630|        734.07|
|   BD-11500|       1102.74|
|   SW-20275|        332.76|
|   AH-10690|       1239.86|
|   JF-15415|        624.41|
|   PH-18790|         47.89|
|   PW-19240|       1233.03|
|   IM-15070|        553.89|
|   KF-16285|       1664.29|
|   NW-18400|       1007.85|
|   OT-18730|       -925.56|
|   JH-15985|       -644.48|
|   KD-16615|        539.72|
|   MG-18145|         82.82|
|   SM-20320|      -1978.12|
+-----------+--------------+
only showing top 20 rows



Spark SQL Queries

In [282]:
order_df.createOrReplaceTempView('orders')

Using SQL output the following aggregates
Profit by Year
Profit by Year + Product Category
Profit by Customer
Profit by Customer + Year


In [ ]:
spark.sql('select  round(sum(profit),2) as yearly_profit from orders group by  year ').show()

In [ ]:
spark.sql('select round(sum(profit),2) as Total_profit from orders group by year, category').show()

In [ ]:
spark.sql('select round(sum(profit),2)  as customer_profit from orders group by customer_id ').show(5)

In [ ]:
spark.sql('select round(sum(profit),2) as Totalprofit from orders group by customer_id, year').show(5)



Only loading customer_df.write
.mode('overwrite')
.format('parquet')
.partitionBy('city')
.option('path','product')
.save()

product.write
.format('parquet')
.mode('overwrite')
.partitionBy('state')
.option('path','product')
.save()

product.write
.format('parquet')
.mode('overwrite')
.partitionBy('order_date')
.bucketBy(4,'customer_id')
.option('path','product')
.save()

Open for suggestions and improvement

********************************************* Thanks ********************************************************